# Analysing BLAST results with BioPython
Now let's take our BLAST results, do a little bit of filtering and generate a multi-FASTA file that we can use as input for generating a MSA

## Installation and Loading
We have installed BioPython in the previous notebook, not need to do it again. In this notebook we make use of the Entrez and NCBIXML modules from BioPython. For documentation see [Entrex](https://biopython.org/docs/1.81/api/Bio.Entrez.html) and [NCBIXML](https://biopython.org/docs/1.81/api/Bio.Blast.NCBIXML.html) 

In [ ]:
# load XML-BLAST parser and Entrez modules
from Bio.Blast import NCBIXML
from Bio import Entrez

Let's open the BLAST results we stored earlier.

In [ ]:
# create file object for reading
blastfile = open("Phospho1_blast.xml")
blastfile

Now we have access to the BLAST results file we could of course write our own parser. Fine if you want to learn programming and want to get coding practice, but not very efficient. The BioPython authors have already solved this problem for you, and (hopefully) dealt with all the quirks and file inconsistencies you ocasionally get from NCBI.  

In [ ]:
# generate BLAST record object
blast_record = NCBIXML.read(blastfile)
blast_record

# if you have results from multiple query sequences you would use NCBIXML.parse to generate an iterable object
### blast_records = NCBIXML.parse(result)


Now we are ready to use the BLAST record object we have just stored. Before we do this, let's create a list to store all hits, and an E-value cutoff for filtering

In [ ]:
# need a list for retrieving all hits as FASTA seqs from NCBI
hits = []
EXPECT_cutoff = 100 # adjust as required

Our BLAST record object contains all the hits from our BLAST search. To process this we need to loop over each hit. Let's print out a few features of each hit. And let's store the identifier of each hit in a list.

In [ ]:
# loop through hsps in alignments, populate hits list (and print out a few bits)
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect <= EXPECT_cutoff:
            print("hit:", alignment.title)
            print("length:", alignment.length)
            print("score:", hsp.score) 
            print("e_value:", hsp.expect)
# It would be useful to have direct access to sequence identifiers, but unfortunately this is not implemented in BioPython. Long story ...
# But we can isolate the identifier by ...
            seq_components = alignment.title.split("|")
            print("id", seq_components[1])
# and we use this to populate list needed for sequence download later
            hits.append(seq_components[1])


The last two hits are clearly (?!) not Phospho1 homologs, so let's adjust the E-value cut-off and re-run the last two blocks.  

Let's get ready for the actual sequence retrieval. Basically we send a list to identifiers to the NCBI, and ask them to send the full FASTA sequences back. 

In [ ]:
# need to set a value  for email to avoid warning. Use your own (or don't).
Entrez.email = "B.Johnson@suspended.com"  # Always tell NCBI who you are
# ... and set name for output file 
filename = "Phospho1_forMSA.fasta"
# let's doublecheck our hitlist
hits

In [ ]:
## need Entrez object to get data and file object to save FASTA sequences of all hits for MSA

net_handle = Entrez.efetch(db="protein", id=hits, rettype="fasta", retmode="text")
out_handle = open(filename, "w")
out_handle.write(net_handle.read())
out_handle.close()
net_handle.close()


Check your outputs - has the file been generated?

In [ ]:
!ls -ltr

Has it worked?

In [ ]:
!cat Phospho1_forMSA.fasta

And we are ready for generating MSAs.